In [ ]:
import pandas as  pd
import seaborn as sns
import matplotlib as plt
df=pd.read_csv('boat_data.csv')
df.dropna()

In [40]:
# df.corr()
# Price conversion --> to EUR
# 1 CHF (Schweize Franken) = 1,02 EUR
# 1 Â£ = 1.15 EUR
# 1 DKK (Dänische Krone)= 0.13 EUR

# Split Currency and Amount from each other
df['Currency']=df['Price'].str[:3]
df['Amount']=df['Price'].str[4:10].str.strip().astype(int)
# Convert to EUR
cur_dict={
    'CHF':1.02,'Â£ ':1.15, 'DKK':0.13,'EUR':1
}
df.replace({'Currency':cur_dict},inplace=True)
df['Price_EUR']=df['Currency']*df['Amount']
df.sample(20)

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days,Currency,Amount,Price_EUR
6234,EUR 115000,Deck Boat,Jeanneau power boats,"Used boat,Unleaded",2018,9.12,2.98,PVC,France,28,1.00,115000,115000.00
9852,EUR 5950,Classic,Glastron power boats,"Used boat,Unleaded",1974,5.20,2.06,GRP,Germany,345,1.00,5950,5950.00
2958,EUR 530000,Flybridge,Ferretti power boats,"Used boat,Diesel",2002,22.33,5.48,GRP,Germany,52,1.00,530000,530000.00
1614,CHF 93000,Sport Boat,Salpa power boats,new boat from stock,2018,7.60,2.53,GRP,Switzerland,197,1.02,93000,94860.00
5674,CHF 168241,Sport Boat,Windy power boats,"new boat from stock,Unleaded",2019,7.88,2.46,GRP,Switzerland,301,1.02,168241,171605.82
8077,EUR 46500,Bowrider,Quicksilver (Brunswick Marine) power boats,"new boat on order,Unleaded",2020,6.20,2.34,GRP,Germany,37,1.00,46500,46500.00
1285,EUR 65000,Motor Yacht,BÃ©nÃ©teau power boats,Used boat,2005,9.80,3.11,GRP,Italy,129,1.00,65000,65000.00
7276,EUR 68000,Sport Boat,Gobbi power boats,"Used boat,Diesel",2000,11.20,3.50,NaN,Italy,76,1.00,68000,68000.00
7642,CHF 62500,"Classic,Runabout",Boesch power boats,"Used boat,Unleaded",1972,5.80,2.10,Wood,Switzerland,300,1.02,62500,63750.00
4746,EUR 280000,Motor Yacht,Carnevali power boats,Used boat,2003,14.90,4.56,NaN,Italy,53,1.00,280000,280000.00


In [53]:
# Categorize Prices
df['Price_EUR'].describe()
# 25th percentile 40,000 EUR
# 50th percentile 87,720 EUR --> median
# 75th percentile 199,937 EUR
# max= 1,144,250
# Categories= Under 45K (0-45K) / Cheap (45K-160K) / Average (160K-220K)/ Expensive(220K-500K)/ Very Expensive  (500K and above)
price_category=pd.cut(df['Price_EUR'],bins=[0,45001,160001,220001,500001,1144250],labels=
['Under 45K','Cheap','Average','Expensive','Very Expensive']) 
df.insert(1,'EUR Price Group',price_category)


ValueError: cannot insert EUR Price Group, already exists

In [55]:
df.sample(20)

,Price,EUR Price Group,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days,Currency,Amount,Price_EUR
3197,EUR 1250000,Cheap,Flybridge,Azimut power boats,Diesel,2010,22.60,5.85,NaN,Greece,65,1.00,125000,125000.0
6336,EUR 109000,Cheap,Flybridge,Riviera power boats,"Used boat,Diesel",2001,12.48,4.02,GRP,France,100,1.00,109000,109000.0
6384,EUR 103718,Cheap,Sport Boat,Invictus power boats,"new boat on order,Diesel",2020,8.70,2.70,GRP,Switzerland,146,1.00,103718,103718.0
2608,EUR 279000,Expensive,Motor Yacht,Cayman power boats,"Used boat,Diesel",2007,17.00,4.65,GRP,Croatia,26,1.00,279000,279000.0
595,EUR 30500,Under 45K,Cabin Boat,Quicksilver power boats,"new boat from stock,Unleaded",2020,5.50,2.29,NaN,France,92,1.00,30500,30500.0
7520,EUR 60000,Cheap,Working Boat,NaN,Used boat,1949,67.04,8.26,NaN,Belgium,336,1.00,60000,60000.0
1336,CHF 72900,Cheap,Sport Boat,Cormate power boats,"Display Model,Unleaded",2017,7.10,2.26,GRP,Switzerland,163,1.02,72900,74358.0
1281,CHF 69500,Cheap,Sport Boat,Viper power boats,"new boat from stock,Unleaded",2020,6.95,2.55,GRP,Switzerland,166,1.02,69500,70890.0
7479,EUR 61055,Cheap,Deck Boat,Quicksilver power boats,"new boat from stock,Unleaded",0,7.88,2.55,GRP,Germany,111,1.00,61055,61055.0
3686,EUR 950000,Very Expensive,Flybridge,Fairline power boats,"Used boat,Diesel",2013,19.35,4.85,GRP,Spain,60,1.00,950000,950000.0
